In [23]:
# ================== Patch AdvancedStacker for backward compatibility, restart & test ==================
import os, time, json, requests, textwrap, sys, types
from pathlib import Path

print("🔧 Patching advanced_stacker.py with legacy-attribute compatibility...")

advanced_stacker_code = r'''
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge

class AdvancedStacker(BaseEstimator, RegressorMixin):
    """
    Backward-compatible stacker that tolerates different internal attribute names
    across training and inference (e.g., base_models_, models_, fitted_base_models).
    """

    def __init__(self, base_models=None, meta_model=None, cv_folds=5):
        self.base_models = base_models if base_models else []
        self.meta_model = meta_model if meta_model else Ridge(alpha=1.0)
        self.cv_folds = cv_folds

    # -------------------- legacy mapping helpers --------------------
    _BASE_KEYS_CANDIDATES = (
        "base_models_", "models_", "fitted_base_models", "base_models_fitted_",
        "estimators_", "estimators", "learners_", "learners"
    )

    _META_KEYS_CANDIDATES = (
        "meta_model_", "meta_model", "meta_", "final_estimator_", "final_estimator",
        "meta", "final_model_", "final_model"
    )

    def _coerce_list(self, obj):
        """Coerce possible structures (dict, list of tuples) to a plain list of estimators."""
        if obj is None:
            return None
        # sklearn-like list of (name, estimator)
        if isinstance(obj, (list, tuple)) and len(obj) > 0 and isinstance(obj[0], (list, tuple)) and len(obj[0]) == 2:
            return [est for _, est in obj]
        # dict of name->estimator
        if isinstance(obj, dict):
            return list(obj.values())
        # already a list/tuple of estimators
        if isinstance(obj, (list, tuple)):
            return list(obj)
        # single estimator
        return [obj]

    def _get_base_models_fitted(self):
        # Preferred, if present
        if hasattr(self, "base_models_") and getattr(self, "base_models_") is not None:
            return self._coerce_list(getattr(self, "base_models_"))
        # Try legacy keys
        for key in self._BASE_KEYS_CANDIDATES:
            if hasattr(self, key):
                val = getattr(self, key)
                if val is not None:
                    return self._coerce_list(val)
        # As a last resort, sometimes base_models (unfitted) were kept but actually contain fitted objects
        if hasattr(self, "base_models") and self.base_models:
            return self._coerce_list(self.base_models)
        return None

    def _get_meta_fitted(self):
        # Preferred
        if hasattr(self, "meta_model_") and getattr(self, "meta_model_") is not None:
            return getattr(self, "meta_model_")
        # Try legacy keys
        for key in self._META_KEYS_CANDIDATES:
            if hasattr(self, key):
                val = getattr(self, key)
                if val is not None:
                    return val
        # Fallback to declared meta_model (may already be fitted in pickle)
        if hasattr(self, "meta_model") and self.meta_model is not None:
            return self.meta_model
        return None

    def __setstate__(self, state):
        """
        On unpickle, map legacy keys onto the modern ones if they exist.
        """
        # Map possible legacy keys → canonical names
        mappings = {
            "models_": "base_models_",
            "fitted_base_models": "base_models_",
            "base_models_fitted_": "base_models_",
            "estimators_": "base_models_",
            "estimators": "base_models_",
            "learners_": "base_models_",
            "learners": "base_models_",

            "meta": "meta_model_",
            "meta_": "meta_model_",
            "final_estimator_": "meta_model_",
            "final_estimator": "meta_model_",
            "final_model_": "meta_model_",
            "final_model": "meta_model_",
        }
        # Copy to avoid mutating while iterating
        new_state = dict(state)
        for old, new in mappings.items():
            if old in new_state and new not in new_state:
                new_state[new] = new_state[old]
        self.__dict__.update(new_state)

    # -------------------- standard API --------------------
    def fit(self, X, y):
        if not self.base_models:
            self.base_models = [
                RandomForestRegressor(n_estimators=100, random_state=42),
                GradientBoostingRegressor(n_estimators=100, random_state=42),
                Ridge(alpha=1.0)
            ]
        # Train base models
        self.base_models_ = [m.fit(X, y) for m in self.base_models]
        # Meta-features
        meta_features = np.column_stack([m.predict(X) for m in self.base_models_])
        # Train meta-model
        self.meta_model_ = self.meta_model.fit(meta_features, y)
        return self

    def predict(self, X):
        # Resolve fitted base models robustly
        base_models_fitted = self._get_base_models_fitted()
        if not base_models_fitted:
            raise AttributeError("AdvancedStacker: no fitted base models found (base_models_)")
        # Build meta features
        meta_features = np.column_stack([m.predict(X) for m in base_models_fitted])
        # Resolve fitted meta
        meta = self._get_meta_fitted()
        if meta is None or not hasattr(meta, "predict"):
            raise AttributeError("AdvancedStacker: no fitted meta model found (meta_model_)")
        return meta.predict(meta_features)

    def get_params(self, deep=True):
        return {'base_models': self.base_models, 'meta_model': self.meta_model, 'cv_folds': self.cv_folds}

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self
'''

Path("advanced_stacker.py").write_text(advanced_stacker_code, encoding="utf-8")
print("✅ advanced_stacker.py updated.")

# Restart uvicorn
print("🔁 Restarting FastAPI server...")
!pkill -f uvicorn 2>/dev/null || true
!fuser -k 8000/tcp 2>/dev/null || true
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &
time.sleep(6)

# Re-check health
print("🩺 Checking /health …")
r = requests.get("http://127.0.0.1:8000/health", timeout=10)
print("status:", r.status_code)
print(json.dumps(r.json(), indent=2))

# Try a prediction again
sample = {
    "timestamp": "2025-01-15T08:30:00",
    "current_light": "Green",
    "eta_to_light_s": 45.5,
    "distance_to_next_light_m": 200.0,
    "vehicle_count": 3,
    "pedestrian_count": 2,
    "lead_vehicle_speed_kmh": 45.0,
    "speed_limit_kmh": 50.0
}
resp = requests.post("http://127.0.0.1:8000/predict", json=sample, timeout=15)
print("\n🔮 /predict status:", resp.status_code)
print("content-type:", resp.headers.get("content-type"))
print("body:", textwrap.shorten(resp.text, width=800, placeholder="…"))
try:
    print("json:", resp.json())
except Exception:
    pass


🔧 Patching advanced_stacker.py with legacy-attribute compatibility...
✅ advanced_stacker.py updated.
🔁 Restarting FastAPI server...
^C
🩺 Checking /health …
status: 200
{
  "status": "healthy",
  "model_loaded": true,
  "features_loaded": true,
  "features_count": 55,
  "message": "Ready",
  "last_error": null,
  "versions": {
    "python": "3.12.12",
    "numpy": "2.0.2",
    "pandas": "2.2.2",
    "joblib": "1.5.2",
    "platform": "Linux-6.6.105+-x86_64-with-glibc2.35"
  }
}

🔮 /predict status: 200
content-type: application/json
body: {"prediction_id":"pred_1763047900","seconds_to_next_change":10.03,"message":"Predicted 10.03 seconds until next light change","features_used":55}
json: {'prediction_id': 'pred_1763047900', 'seconds_to_next_change': 10.03, 'message': 'Predicted 10.03 seconds until next light change', 'features_used': 55}


In [24]:
import requests, textwrap

URL = "http://127.0.0.1:8000/predict"   # if you were hitting the Cloudflare URL, switch to localhost first
payload = {
    "timestamp": "2025-11-13T15:45:00",
    "current_light": "Green",
    "eta_to_light_s": 100.0,
    "distance_to_next_light_m": 150.0,
    "vehicle_count": 3,
    "pedestrian_count": 1,
    "lead_vehicle_speed_kmh": 42.0,
    "speed_limit_kmh": 50.0
}

r = requests.post(URL, json=payload, timeout=15)

print("status:", r.status_code)
print("content-type:", r.headers.get("content-type"))
print("raw text (first 800 chars):\n", textwrap.shorten(r.text, width=800, placeholder="…"))

# Only call .json() if it actually looks like JSON
if r.headers.get("content-type","").lower().startswith("application/json"):
    print("as json:", r.json())
else:
    print("Not JSON; see raw text above.")


status: 200
content-type: application/json
raw text (first 800 chars):
 {"prediction_id":"pred_1763047928","seconds_to_next_change":10.02,"message":"Predicted 10.02 seconds until next light change","features_used":55}
as json: {'prediction_id': 'pred_1763047928', 'seconds_to_next_change': 10.02, 'message': 'Predicted 10.02 seconds until next light change', 'features_used': 55}
